In [ ]:
# 미리 다운로드 받아야 할 파일
#caffe : res10_300x300_ssd_iter_140000_fp16.caffemodel, deploy.prototxt
#tensorflow : opencv_face_detector_uint8.pb, opencv_face_detector.pbtxt

import os
import cv2
import sys 
import numpy as np

# caffe, tensorflow 모델을 차례로 테스트 해볼것
model = "../models/ssd/res10_300x300_ssd_iter_140000_fp16.caffemodel" #다운로드 링크 : https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel
config = "../models/ssd/deploy.prototxt" #다운로드 링크 : https://github.com/opencv/opencv/blob/master/samples/dnn/face_detector/deploy.prototxt
#model = "opencv_face_detector_uint8.pb" #다운로드 링크 : https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20180220_uint8/opencv_face_detector_uint8.pb
#config = "opencv_face_detector.pbtxt" #다운로드 링크 : https://github.com/opencv/opencv/blob/master/samples/dnn/face_detector/opencv_face_detector.pbtxt

files = [model, config]

#설정파일 있는지 체크
for f in files:
    if os.path.isfile(f) is False:
        print("모델 혹은 설정파일이 없습니다 : "+f)
        sys.exit()

cap = cv2.VideoCapture(0)   #카메라 오픈
if not cap.isOpened():
    print("Camera open failed!")
    sys.exit()

net = cv2.dnn.readNet(model, config)  #모델 불러오기
if net.empty():
    print("Net open failed!")
    sys.exit()

while True:
    ret, frame = cap.read()   #카메라 프레임 읽어오기

    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123)) #이미지 크기, 컬러 평균값 지정
    net.setInput(blob) #입력 설정
    out = net.forward() #추론

    detect = out[0, 0, :, :]   # 출력 배열에서 뒷쪽 2개 차원만 활용  [200,7]
    (h, w) = frame.shape[:2]  #카메라 프레임의 크기 읽어오기 

    for i in range(detect.shape[0]):     #200행을 차례로 불러오기
        confidence = detect[i, 2]          # c 값을 읽어 confidence에 저장
        if confidence < 0.5:  #confidence 가 0.5보다 작을때는 skip 
            break
        x1 = int(detect[i, 3] * w)  #현재 프레임에 맞춰서 좌표 계산
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))  # 박스 그리기
        label = f"Face: {confidence:4.2f}"    #확률값 출력 
        cv2.putText(frame, label, (x1, y1 - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frame", frame)  #출력

    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
